In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

import numpy as np
import pandas as pd
import math
import glob
import time
import pickle

import steering
import speeding

import logging

In [2]:
PROJECT_ROOT = '../../../..'
LEARNED_DRIVER = 'snakeoil_miner/data'
DIFFICULTY = ['easy', 'medium', 'hard']

# COMMAND = 'steering'
COMMAND = 'speeding'
logging.basicConfig(filename='logs/{}/training-{}.log'.format(COMMAND, time.time()),level=logging.DEBUG)

NUM_EPOCHS = 400

TRAINING_FILES = []
for d in DIFFICULTY:
    TRAINING_FILES.extend(glob.glob('/'.join([PROJECT_ROOT, LEARNED_DRIVER, d, '*.csv'])))

TRAINING_DATA = {}
for FILE in TRAINING_FILES:
    DF = pd.read_csv(FILE, index_col=False)
    TRAINING_DATA[FILE] = DF.values
    
CUDA = torch.cuda.is_available()
if CUDA:
    DTYPE = torch.cuda.FloatTensor
else:
    DTYPE = torch.FloatTensor

In [3]:
def save_checkpoint(state, is_best, filepath='latest_checkpoint.tar'):
    torch.save(state, 'split_checkpoints/' + COMMAND + '/' + filepath)
    if is_best:
        torch.save(state, 'split_checkpoints/' + COMMAND + '/' + 'best_checkpoint.tar')

In [4]:
def train(training_data, model, criterion, optimzier):
    loss = 0
    model.train(mode=True)
    for key in training_data:
        logging.info('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        print('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        
        model.hidden_state = model.init_hidden()
        optimizer.zero_grad()
        track_sequence = training_data[key]

        if COMMAND == 'steering':
            targets = track_sequence[:, 2:3]
        elif COMMAND == 'speeding':
            targets = track_sequence[:, 0:2]
        
        inputs = track_sequence[:, 3:]

        targets_variable = autograd.Variable(torch.Tensor(targets)).type(DTYPE)
        inputs_variable = autograd.Variable(torch.Tensor(inputs),  requires_grad=True).type(DTYPE)

        outputs_variable = model(inputs_variable)

        track_loss = criterion(outputs_variable, targets_variable)

        track_loss.backward()
        optimizer.step()

        loss += track_loss.data[0]
    return loss

In [5]:
def validate(validation_data, model, criterion):
    loss = 0
    model.train(mode=False)
    for key in validation_data:
        logging.info('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        print('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        
        model.hidden_state = model.init_hidden()
        track_sequence = validation_data[key]

        if COMMAND == 'steering':
            targets = track_sequence[:, 2:3]
        elif COMMAND == 'speeding':
            targets = track_sequence[:, 0:2]
            
        inputs = track_sequence[:, 3:]

        targets_variable = autograd.Variable(torch.Tensor(targets), volatile=True).type(DTYPE)
        inputs_variable = autograd.Variable(torch.Tensor(inputs), volatile=True).type(DTYPE)

        outputs_variable = model(inputs_variable)

        track_loss = criterion(outputs_variable, targets_variable)

        loss += track_loss.data[0]
    return loss

In [6]:
checkpoint = torch.load('split_checkpoints/{}/latest_checkpoint.tar'.format(COMMAND))
if COMMAND == 'steering':
    model = steering.Steering(steering.HYPERPARAMS.INPUT_SIZE,
                               steering.HYPERPARAMS.LSTM_HIDDEN_SIZE,
                               steering.HYPERPARAMS.HIDDEN_LAYER_SIZE,
                               steering.HYPERPARAMS.DROPOUT_PROB,
                               steering.HYPERPARAMS.NUM_LAYERS,
                               steering.HYPERPARAMS.TARGET_SIZE,
                               steering.HYPERPARAMS.BATCH_SIZE)
    optimizer = optim.Adam(model.parameters(), lr=steering.HYPERPARAMS.LEARNING_RATE)
elif COMMAND == 'speeding':
    model = speeding.Speeding(speeding.HYPERPARAMS.INPUT_SIZE,
                               speeding.HYPERPARAMS.LSTM_HIDDEN_SIZE,
                               speeding.HYPERPARAMS.HIDDEN_LAYER_SIZE,
                               speeding.HYPERPARAMS.DROPOUT_PROB,
                               speeding.HYPERPARAMS.NUM_LAYERS,
                               speeding.HYPERPARAMS.TARGET_SIZE,
                               speeding.HYPERPARAMS.BATCH_SIZE)
    optimizer = optim.Adam(model.parameters(), lr=speeding.HYPERPARAMS.LEARNING_RATE)
model.load_state_dict(checkpoint['state_dict'])

if CUDA:
    model.cuda()
        
criterion = nn.MSELoss().cuda()

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', verbose=True)

min_loss = checkpoint['min_loss']
losses = {
  'training': [],
  'validation': []
}

In [7]:
logging.info('Training {}...'.format(COMMAND))
print('Training {}...'.format(COMMAND))
for epoch in np.arange(checkpoint['epoch'], NUM_EPOCHS):
    logging.info('Epoch [%d/%d]' %(epoch+1, NUM_EPOCHS))
    print('Epoch [%d/%d]' %(epoch+1, NUM_EPOCHS))
    
    is_best = False

    training_loss = train(TRAINING_DATA, model, criterion, optimizer)
    logging.info('--- TRAINING LOSS: %f' % training_loss)
    print('--- TRAINING LOSS: %f' % training_loss)

#     validation_loss = validate(VALIDATION_DATA, model, criterion)
#     logging.info('--- VALIDATION LOSS: %f' % validation_loss)
#     print('--- VALIDATION LOSS: %f' % validation_loss)

    if training_loss < min_loss:
        logging.info('--- --- best model found so far: %f' % training_loss)
        print('--- --- best model found so far: %f' % training_loss)
        min_loss = training_loss
        is_best = True

    losses['training'].append(training_loss)
#     losses['validation'].append(validation_loss)

    save_checkpoint({
          'epoch': epoch + 1,
          'state_dict': model.state_dict(),
          'min_loss': min_loss,
          'optimizer' : optimizer.state_dict(),
      }, is_best)

    scheduler.step(training_loss)
    logging.info('-------------------------------------------------------')
    print('-------------------------------------------------------')

Training speeding...
Epoch [201/400]
--- Parsing track medium-race_13.csv
--- Parsing track hard-race_21.csv
--- Parsing track hard-race_23.csv
--- Parsing track medium-race_12.csv
--- Parsing track easy-race_03.csv
--- Parsing track medium-race_11.csv
--- Parsing track easy-race_02.csv
--- Parsing track medium-race_10.csv
--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 1.026719
-------------------------------------------------------
Epoch [202/400]
--- Parsing track medium-race_13.csv
--- Parsing track hard-race_21.csv
--- Parsing track hard-race_23.csv
--- Parsing track medium-race_12.csv
--- Parsing track easy-race_03.csv
--- Parsing track medium-race_11.csv
--- Parsing track easy-race_02.csv
--- Parsing track medium-race_10.csv
--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRA

--- Parsing track medium-race_10.csv
--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 0.616652
-------------------------------------------------------
Epoch [217/400]
--- Parsing track medium-race_13.csv
--- Parsing track hard-race_21.csv
--- Parsing track hard-race_23.csv
--- Parsing track medium-race_12.csv
--- Parsing track easy-race_03.csv
--- Parsing track medium-race_11.csv
--- Parsing track easy-race_02.csv
--- Parsing track medium-race_10.csv
--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 0.609976
-------------------------------------------------------
Epoch [218/400]
--- Parsing track medium-race_13.csv
--- Parsing track hard-race_21.csv
--- Parsing track hard-race_23.csv
--- Parsing track medium-race_12.csv
--- Parsing track easy-race_03.csv
--- Parsing trac

--- Parsing track hard-race_21.csv
--- Parsing track hard-race_23.csv
--- Parsing track medium-race_12.csv
--- Parsing track easy-race_03.csv
--- Parsing track medium-race_11.csv
--- Parsing track easy-race_02.csv
--- Parsing track medium-race_10.csv
--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 0.527368
-------------------------------------------------------
Epoch [233/400]
--- Parsing track medium-race_13.csv
--- Parsing track hard-race_21.csv
--- Parsing track hard-race_23.csv
--- Parsing track medium-race_12.csv
--- Parsing track easy-race_03.csv
--- Parsing track medium-race_11.csv
--- Parsing track easy-race_02.csv
--- Parsing track medium-race_10.csv
--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 0.526632
-----------------------------------------------------

--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 0.487162
-------------------------------------------------------
Epoch [248/400]
--- Parsing track medium-race_13.csv
--- Parsing track hard-race_21.csv
--- Parsing track hard-race_23.csv
--- Parsing track medium-race_12.csv
--- Parsing track easy-race_03.csv
--- Parsing track medium-race_11.csv
--- Parsing track easy-race_02.csv
--- Parsing track medium-race_10.csv
--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 0.485471
-------------------------------------------------------
Epoch [249/400]
--- Parsing track medium-race_13.csv
--- Parsing track hard-race_21.csv
--- Parsing track hard-race_23.csv
--- Parsing track medium-race_12.csv
--- Parsing track easy-race_03.csv
--- Parsing track medium-race_11.csv
--- Parsing track easy-race_02.csv
--- Parsing trac

--- Parsing track hard-race_23.csv
--- Parsing track medium-race_12.csv
--- Parsing track easy-race_03.csv
--- Parsing track medium-race_11.csv
--- Parsing track easy-race_02.csv
--- Parsing track medium-race_10.csv
--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 0.457298
-------------------------------------------------------
Epoch [264/400]
--- Parsing track medium-race_13.csv
--- Parsing track hard-race_21.csv
--- Parsing track hard-race_23.csv
--- Parsing track medium-race_12.csv
--- Parsing track easy-race_03.csv
--- Parsing track medium-race_11.csv
--- Parsing track easy-race_02.csv
--- Parsing track medium-race_10.csv
--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 0.452884
-------------------------------------------------------
Epoch [265/400]
--- Parsing trac

...

--- Parsing track medium-race_12.csv
--- Parsing track easy-race_03.csv
--- Parsing track medium-race_11.csv
--- Parsing track easy-race_02.csv
--- Parsing track medium-race_10.csv
--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 0.407616
-------------------------------------------------------
Epoch [341/400]
--- Parsing track medium-race_13.csv
--- Parsing track hard-race_21.csv
--- Parsing track hard-race_23.csv
--- Parsing track medium-race_12.csv
--- Parsing track easy-race_03.csv
--- Parsing track medium-race_11.csv
--- Parsing track easy-race_02.csv
--- Parsing track medium-race_10.csv
--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 0.405276
-------------------------------------------------------
Epoch [342/400]
--- Parsing track medium-race_13.csv
--- Parsing tr

--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 0.385848
-------------------------------------------------------
Epoch [356/400]
--- Parsing track medium-race_13.csv
--- Parsing track hard-race_21.csv
--- Parsing track hard-race_23.csv
--- Parsing track medium-race_12.csv
--- Parsing track easy-race_03.csv
--- Parsing track medium-race_11.csv
--- Parsing track easy-race_02.csv
--- Parsing track medium-race_10.csv
--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 0.387596
-------------------------------------------------------
Epoch [357/400]
--- Parsing track medium-race_13.csv
--- Parsing track hard-race_21.csv
--- Parsing track hard-race_23.csv
--- Parsing track medium-race_12.csv
--- Parsing track easy-race_03.csv
--- Parsing track medium-race_11.csv
--- Parsing trac

--- Parsing track easy-race_02.csv
--- Parsing track medium-race_10.csv
--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 0.365827
-------------------------------------------------------
Epoch [371/400]
--- Parsing track medium-race_13.csv
--- Parsing track hard-race_21.csv
--- Parsing track hard-race_23.csv
--- Parsing track medium-race_12.csv
--- Parsing track easy-race_03.csv
--- Parsing track medium-race_11.csv
--- Parsing track easy-race_02.csv
--- Parsing track medium-race_10.csv
--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 0.365480
--- --- best model found so far: 0.365480
-------------------------------------------------------
Epoch [372/400]
--- Parsing track medium-race_13.csv
--- Parsing track hard-race_21.csv
--- Parsing track hard-race_23.csv
--- Parsing

--- Parsing track easy-race_03.csv
--- Parsing track medium-race_11.csv
--- Parsing track easy-race_02.csv
--- Parsing track medium-race_10.csv
--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 0.362597
-------------------------------------------------------
Epoch [386/400]
--- Parsing track medium-race_13.csv
--- Parsing track hard-race_21.csv
--- Parsing track hard-race_23.csv
--- Parsing track medium-race_12.csv
--- Parsing track easy-race_03.csv
--- Parsing track medium-race_11.csv
--- Parsing track easy-race_02.csv
--- Parsing track medium-race_10.csv
--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 0.362997
-------------------------------------------------------
Epoch [387/400]
--- Parsing track medium-race_13.csv
--- Parsing track hard-race_21.csv
--- Parsing trac

--- Parsing track medium-race_11.csv
--- Parsing track easy-race_02.csv
--- Parsing track medium-race_10.csv
--- Parsing track easy-race_00.csv
--- Parsing track hard-race_20.csv
--- Parsing track easy-race_01.csv
--- Parsing track hard-race_22.csv
--- TRAINING LOSS: 0.360220
-------------------------------------------------------


In [8]:
# dump losses to file
with open('split_checkpoints/{}/losses_{}.pkl'.format(COMMAND, time.time()), 'wb') as file:
    pickle.dump(losses, file)